# Spam Detection with OpenAI

This example showcases how to leverage OpenAI within the transform function and update the events data with the response. 
In this example, we will detect spam comments using OpenAI and add the result to the event. 

The entire process runs as a seververless event driven pipeline on GlassFlow. 

## Pre-requisites

- Create your free GlassFlow account via the [GlassFlow WebApp](https://app.glassflow.dev).
- Get your [Personal Access Token](https://app.glassflow.dev/profile) to authorize the Python SDK to interact with GlassFlow Cloud.
- Get your OpenAI API Key https://platform.openai.com/

In [ ]:
%pip install "glassflow>=2.0.5" pandas

In [ ]:
import glassflow

In [ ]:
# fill credentials
# Please edit this variable with your own personal access token from https://app.glassflow.dev/profile
personal_access_token = ""
OPENAI_API_KEY=""


## Create Pipeline

In [ ]:
client = glassflow.GlassFlowClient(
    personal_access_token=personal_access_token
)

In [ ]:
# Get the space named "examples" (or create one if no space is found)
list_spaces = client.list_spaces()

space_name = "examples"
for s in list_spaces.spaces:
    if s["name"] == space_name:
        space = glassflow.Space(
            personal_access_token=client.personal_access_token,
            id=s["id"], 
            name=s["name"]
        )
        break
else:
    space = client.create_space(name=space_name)

print(f"Space \"{space.name}\" with ID: {space.id}")

### Transformation Function

In [ ]:
%pycat transform.py

### Requirements txt

In [ ]:
with open("requirements.txt") as f:
    requirements_txt = f.read()
print(requirements_txt)

### Environment variables

In [ ]:
env_vars = [{
  "name": "OPENAI_API_KEY",
  "value": OPENAI_API_KEY
}]

### Create Pipeline

In [ ]:
pipeline_name = "spam-detection-openai"

pipeline = client.create_pipeline(
    name=pipeline_name, 
    transformation_file='transform.py',
    space_id=space.id, 
    env_vars=env_vars,
    requirements=requirements_txt
)
print("Pipeline ID:", pipeline.id)

In [ ]:
print("Pipeline is deployed!") 
print("Pipeline Id = %s" % (pipeline.id))
print("Pipeline URL %s "% f"https://app.glassflow.dev/pipelines/{pipeline.id}")

## Produce data and send it to your pipeline

### Get sample data from a public dataset 

In [ ]:
import pandas as pd

def get_data_sample(sample_size=100):
    """
    Fetches and samples the SMS spam dataset.
    """
    url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/sms.tsv'
    df = pd.read_csv(url, sep='\t', header=None, names=['label', 'message'])
    # Sample specified number of negative and positive examples without replacement
    negatives = df[df['label'] == 'ham'].sample(sample_size//2, random_state=42)
    positives = df[df['label'] == 'spam'].sample(sample_size//2, random_state=42)
    df_sampled = pd.concat([negatives, positives]).reset_index(drop=True)
    df_shuffled = df_sampled.sample(frac=1)
    data = df_shuffled.to_dict('records')
    return data

In [ ]:
sample_chat_dataset = get_data_sample()

### Get pipeline data source object to publish events to the pipeline

In [ ]:
data_source = pipeline.get_source()

In [ ]:
# Generate some data and send it to the pipeline. Store it locally to compare
n_events = 10
input_events = []
for event in sample_chat_dataset[0:n_events]:    
    input_events.append(event)
    data_source.publish(event)

### Display data sent to the pipeline

In [ ]:
import pandas as pd

display(pd.DataFrame(input_events))

## Consume events from the pipeline 

Get pipeline data sink to consume the transformed events from the pipeline.

In [ ]:
data_sink = pipeline.get_sink()

In [ ]:
output_events = []
while True:
    resp = data_sink.consume()
    if resp.status_code == 200:
        output_events.append(resp.json())
    if len(output_events) == n_events:
        # all events have been consumed
        break

### Display data received from the pipeline

In [ ]:
import pandas as pd

display(pd.DataFrame(output_events))